In [6]:
import numpy as np
from numpy.linalg import inv

In [7]:
!pip install -r requirements.txt

# Reference Frame

To make sure things are consistent, the reference frame of the robot will match the reference frame of ROS.


**Position for positive x will be up (North), positive y will be to the left (West), and positive z will be out of the page.**


![alternatvie text](reference_frame.png)

# Linear Dynamics
$$\dot{x} = A\vec{x} + B\vec{u} \\
 \vec{y} = C\vec{x}$$

$\vec{x}$ is the state variables. $v_{x}, v_{y}$ is the world linear velocity of the robot in m/s, $\omega$ is the world reference frame angular velocity of the robot around the z-axis in rad/s.
$$
\vec{x} = \begin{bmatrix}
v_{x} \\ 
v_{y} \\
\omega_{z}
\end{bmatrix}
$$

$\vec{u}$ is the input vector. $\omega_{iFL}, \omega_{iBL}, \omega_{iBR}, \omega_{iFR}$ is commanded angular velocity of the front left, front right, back left, back right wheels of the robot (will refer to those positions for the remainder).
$$
\vec{u} = \begin{bmatrix}
\omega_{iFL} \\
\omega_{iBL} \\
\omega_{iBR} \\
\omega_{iFR}
\end{bmatrix}
$$

$\vec{y}$ is the measured observation vector. $\omega_{oFL},\omega_{oBL},\omega_{oBR},\omega_{oFR}$ is the angular velocity for wheels of the robot in rad/s. $\omega_{gyro}$ is the angular velocity of the robot in robot reference frame measured by the gyro in rad/s.

$$
\vec{y} = \begin{bmatrix}
\omega_{oFL} \\
\omega_{oBL} \\
\omega_{oBR} \\
\omega_{oFR} \\
\omega_{\text{gyro}}
\end{bmatrix}
$$

## Body to Wheel Conversion

We need to make a transformation from the robot reference frame to the wheel reference frame
$r_{FL}, r_{BL}, r_{BR}, r_{FR}$ is the wheel radius in m.

$\theta_{FL}, \theta_{BL}, \theta_{BR}, \theta_{FR}$ is the angle offset from the center in rads.

$l_{FL}, l_{BL}, l_{BR}, l_{FR}$ is the distance to center in m.

Based on work here: https://www.mdpi.com/2076-3417/12/12/5798
$$
    \textbf{T}_{Wheel} = 
    \begin{bmatrix}
        -cos(\theta_{FL})/r_{FL} & -sin(\theta_{FL})/r_{FL} & l_{FL}/r_{FL} \\
        -cos(\theta_{BL})/r_{BL} & -sin(\theta_{BL})/r_{BL} & l_{BL}/r_{BL} \\
        -cos(\theta_{BR})/r_{BR} & -sin(\theta_{BR})/r_{BR} & l_{BR}/r_{BR} \\
        -cos(\theta_{FR})/r_{FR} & -sin(\theta_{FR})/r_{FR} & l_{FR}/r_{FR} \\
    \end{bmatrix}
$$

## Wheel to Bot Conversion

Pseudo-inverse of $T_{Wheel}$ is the $T_{Body}$
$$
    \textbf{T}_{Body} = \textbf{T}^{\dagger}_{Wheel} = (\textbf{T}_{Wheel}^{T} \cdot \textbf{T}_{Wheel})^{-1} \textbf{T}_{Wheel}^{T}
$$

# Kalman Filter

In [8]:
process_noise = 0.05 # State noise for unmodeled influences, TODO unclear where is comes from
encoder_sampling_freq = 100 # Hz
encoder_noise_rads = np.radians(0.068) # rads for AS5047P encoder RMS output noise (1 sigma)
encoder_noise = encoder_noise_rads/(1/encoder_sampling_freq) # rads/s TODO unclear if this is valid
gryo_noise_noise_density = 0.007  # 0.014 deg/s/√Hz for BMI085 Noise density (typ.), 0.007 deg/sec/√Hz for BMI323
gyro_sampling_freq = 100 # Hz
gyro_noise = np.radians(gryo_noise_noise_density * np.sqrt(gyro_sampling_freq)) * 1/10000 # in rad/s

intial_covariance = 0.1 # Picked at random TODO

print("Encoder Noise: " + str(encoder_noise))
print("Gyro Noise: " + str(gyro_noise))

Encoder Noise: 0.11868238913561441
Gyro Noise: 1.2217304763960308e-07


In [9]:
import numpy as np
import math
from numpy.linalg import inv
from KalmanFilter import KF
from SteadyStateKalmanFilter import SS_KF
from RobotModel import RobotModel

model = RobotModel()
observer_type = "SS_KF"

A, B, H, D, P, Q, R = model.generate_model().values()
kalmanFilter = KF(
    x_hat_init=np.zeros((3, 1)), A=A, B=B, H=H, D=D, P=P, Q=Q, R=R
)
ss_kalmanFilter = SS_KF(
    x_hat_init=np.zeros((3, 1)), A=A, B=B, H=H, D=D, Q=Q, R=R
)
observer_list = [kalmanFilter, ss_kalmanFilter]
observer = ss_kalmanFilter

print("A = ")
print(A)

print("B = ")
print(B)

print("H = ")
print(H)

print("D = ")
print(D)

print("P = ")
print(P)

print("Q = ")
print(Q)

print("R = ")
print(R)

print("K_ss = ")
print(ss_kalmanFilter.gains.K)

print("K = ")
print(kalmanFilter.gains.K)

A = 
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
B = 
[[-0.0103923  -0.00848528  0.00848528  0.0103923 ]
 [ 0.01242641 -0.01242641 -0.01242641  0.01242641]
 [ 0.1079459   0.07632928  0.07632928  0.1079459 ]]
H = 
[[-28.86751346  16.66666667   2.71333333]
 [-23.57022604 -23.57022604   2.71333333]
 [ 23.57022604 -23.57022604   2.71333333]
 [ 28.86751346  16.66666667   2.71333333]
 [  0.           0.           1.        ]]
D = 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
P = 
[[0.1 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  0.1]]
Q = 
[[1.5e-05 0.0e+00 0.0e+00]
 [0.0e+00 1.5e-05 0.0e+00]
 [0.0e+00 0.0e+00 1.5e-05]]
R = 
[[1.18682389e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 1.18682389e-05 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.18682389e-05 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.18682389e-05
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000

Code below taken from RoboRacing-firmware (written by Peter Wilson)

In [10]:
#Read in the CSV of commanded velocities
import csv
ns_per_sec = 1e9
with open('software_commanded_vel.csv') as velocity_csv:
    velocity_reader = csv.reader(velocity_csv, delimiter=',')
    next(velocity_reader)   #Skip first line, the headers
    software_velocity_times_commands = np.asarray([(int(line[0]), float(line[1]), float(line[2])) for line in velocity_reader])
software_velocity_times_commands[:,0] /= ns_per_sec     #Convert times from ns to seconds
software_velocity_times_commands[:,0] -= software_velocity_times_commands[0,0]   #Make time start at 0

print(software_velocity_times_commands)

#Remember the last time and index we were ordered to fe
last_gotten_idx = 0
last_gotten_time = 0
def get_software_vel(time):
    '''
    Gets the last command software sent at the given time.
    Remembers the index of the last time it was called for increased efficiency in case time is monotonically increasing
    '''
    global last_gotten_idx
    global last_gotten_time

    if time > last_gotten_time:
        idx = last_gotten_idx
    else:
        idx = 0

    while software_velocity_times_commands[idx + 1, 0] < time:
        idx += 1

    last_gotten_idx = idx
    last_gotten_time = time
    return software_velocity_times_commands[idx, 1]

print(get_software_vel(0))

FileNotFoundError: [Errno 2] No such file or directory: 'software_commanded_vel.csv'

In [ ]:
dt = 5e-5  # s. Timestep
end_time = 10 # s
# TODO determine transmission period, ros bag freq, and software to robot transmission delay
sw_transmission_period = 0.003 # s. How often software transmits commands
sw_to_robot_transmission_delay = 0.003 # s. How long it takes from ROS setting a velocity to getting to the robot
sw_to_robot_transmission_delay_index = int(sw_to_robot_transmission_delay/dt)

# On-robot controller parameters (so we can tweak them)
robot_controller_loop_time = 5e-3   # s. How frequently the on robot controller runs

#Define data logging arrays
times, dt = np.linspace(0, end_time, num = round(end_time/dt), retstep=True)  #array of times
actual_state_arr = np.zeros(3,1,len(times)) # x_k -> [v_x, v_y, Omega]
estimated_state_arr = np.zeros(len(times)) # \hat{x}
sw_target_state = np.zeros(3,1,len(times)) # x sent from software
robot_target_state = np.zeros(3,1,len(times)) # x when received on robot
command_input_arr = np.zeros(4,1,len(times)) # u -> [omega_ifl, omega_ibl, omega_ibr, omega_ifr]
measured_state_arr = np.zeros(5,1,len(times)) # y -> [omega_ofl, omega_obl, omega_obr, omega_ofr, omega_gyro]
robot_controller_loop_time = 0.010 # s. How often the robot motion controller is running

#Now write the simulation code
command_wheel_velocities = [0,0,0,0]
last_controller_execution = 0

software_vel_cmd_delay = np.zeros(actual_state_arr.shape)

model = RobotModel()
observer_type = "SS_KF"

A, B, H, D, P, Q, R = model.generate_model().values()
kalmanFilter = KF(
    x_hat_init=np.zeros((3, 1)), A=A, B=B, H=H, D=D, P=P, Q=Q, R=R
)
ss_kalmanFilter = SS_KF(
    x_hat_init=np.zeros((3, 1)), A=A, B=B, H=H, D=D, Q=Q, R=R
)
observer_list = [kalmanFilter, ss_kalmanFilter]
observer = ss_kalmanFilter


for i in range(len(times)):
    # Time is from perspective of the PC, but independent of ROS
    time = times[i]

    # Find software's current velocity command
    sw_target_state = get_software_vel(time)

    # Delay to account for software transmission
    if (i > sw_to_robot_transmission_delay_index):
        robot_target_state = sw_target_state[i - sw_to_robot_transmission_delay_index]


    # Execute the estimator
    if time - last_controller_execution > robot_controller_loop_time:
        timestep = time - last_controller_execution

        kalmanFilter.step(robot_target_state)
        #estimate speed
        kalman_filter_est_vel = estimate_vel(timestep, current_reading, command_brakes, num_encoder_ticks_remembered_by_code)
        num_encoder_ticks_remembered_by_code = 0

        last_controller_execution = time

    #Bound the control signals
    true_voltage, true_motor_current = get_true_voltage_current(car_true_vel, command_voltage)
    true_brake_force = get_true_braking_force(car_true_vel, sent_brake_command)

    #Car physics:
    car_true_position += dt*car_true_vel
    car_true_vel += dt*(-true_gamma_speed*car_true_vel + true_gamma_volt*true_voltage - true_gamma_brake*true_brake_force)

    #encoder callback interrupt fires
    if total_num_encoder_ticks/magnets_per_m < car_true_position:
        total_num_encoder_ticks += 1
        num_encoder_ticks_remembered_by_code += 1

    #Now do the logs
    vel_kart_arr[i] = car_true_vel
    estimated_vel_arr[i] = get_speed()
    filtered_vel_arr[i] = get_curr_target_speed()
    software_command_velocities_at_timestep_arr[i] = software_vel_cmd
    voltage_cmd_arr[i] = command_voltage
    brake_cmd_arr[i] = command_brakes
    voltage_realized_arr[i] = true_voltage
    brake_realized_arr[i] = true_brake_force
    motor_current_arr[i] = true_motor_current
    error_integral_arr[i] = get_error_integral()